# Metrics in EdnaML and EdnaDeploy

Here, we examine EdnaML's metrics infrastructure. Most metrics are essentially wrappers around their Torchmetrics 

# Setup Steps (If EdnaML is not already installed)
 
We can install either from source or from PyPi. The appropriate option can be selected from the first cell below.

**Very Important**. Due to the way Colab installs certain packages, you will need to restart the runtime after installing EdnaML. Then you can proceed with future steps.

In [ ]:
!nvidia-smi

In [ ]:
install_from = "source" # source | pypi
branch = "suprem-devel-mongo-metrics"           # DO NOT CHANGE THIS unless you know what you are doing
version = "0.1.5"           # DO NOT CHANGE THIS unless you know what you are doing

###  Installation steps

In [ ]:
if install_from == "source":
  ! rm -rf -- EdnaML ||:
  ! git clone -b $branch https://github.com/asuprem/EdnaML
  ! pip install -e EdnaML/
else:
  ! python -V
  ! pip3 install --pre ednaml==$version

## Restart Runtime

In [ ]:
try:
  import ednaml
except (ImportError, KeyError, ModuleNotFoundError):
  print('Stopping RUNTIME. Colaboratory will restart automatically.')
  exit()

# Metrics Basics

We will run a few experiments using the basic [MNIST configuration](./mnist.yml). See [cnn.ipynb](../0-basics/cnn/cnn.ipynb) for more details on MNIST and EdnaML.

We have added a few things here for Metrics on top of the Storage and experimentation components. Specifically, we have added the `METRICS` section with a single metric:

```
METRICS:                      # Defining metrics to be tracked. This needs to be made top-level
  MODEL_METRICS:
    - METRIC_NAME: avgacc
      METRIC_CLASS: BaseTorchMetric
      METRIC_ARGS:
        metric_name: Accuracy
        aggregate: 100
        metric_kwargs:
          task: 'multiclass'
          num_classes: 10
      METRIC_PARAMS: 
        preds: logits
        targets: labels  # basically, key is what the metric expects, value is what WILL be there, e.g. HFTrainer could have labels, not targets, so it would be targets: labels
      METRIC_TRIGGER: step
      METRIC_STORAGE: null
```

Here, we have set up a Metric for the model (more details about specific metrics can be found at []). You can add metrics for each component to track different KPIs, e.g. metrics for the Logger, Cnfiguration manager, Deployment, and Code. For example you may want to record code quality of provided custom code through some backend service as a metric. You may also want to record number of error logs, warning logs, or debug logs per X training steps. 

In this case, we have added an Accuracy Metric. Since we want to use the excellent Torchmetrics package, we will use our internal wrapper `BaseTorchMetric`, and provide the correct arguments (`metric_name` is the specific torchmetric we wish to use, and `metric_kwargs` are the arguments for the above class). 

Then, we have `METRIC_PARAMS`. Each module publishes a set of parameters internally that metrics have access to. For example, `BaseTrainer` publishes `loss` at each step. `ClassificationTrainer`, which we use for MNIST, publishes loss, as well as `logits`, `labels`, `features` and the current `epoch` and `step`, among others. For accuracy, we require `logits` and `labels`. However, `torchmetrics.Accuracy` takes in `preds` and `targets` as input. So we provide this mapping in `METRIC_PARAMS` so that our metrics can access the correct arguments to compute metrics.

Finally, we have `METRIC_TRIGGER`, which can be one of `[once | always | step | batch]`, meaning it is triggered just once at beginning, always whenever some parameter changes inside a module, at the end of each step, or at the end of a batch of steps (batch determined by `LOGGING.STEP_VERBOSE`). We want to compute accuracy at each step. However, we also want to aggregate the accuracy for saving across 100 steps, so we set the `aggregate` parameter in `METRIC_ARGS` to 100.



## 0. Setting up the MNIST model

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Here we define our custom model class
from ednaml.models import ModelAbstract
from torch import nn
import ednaml.core.decorators as edna

class MNISTModel(ModelAbstract):
  def model_attributes_setup(self, **kwargs):
    pass
  def model_setup(self, **kwargs):
    self.conv1 = nn.Sequential(         
        nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2), 
        nn.ReLU(), 
        nn.MaxPool2d(kernel_size=2),    
    )
    self.conv2 = nn.Sequential(         
        nn.Conv2d(16, 32, 5, 1, 2), nn.ReLU(), nn.MaxPool2d(2),                
    )
    # fully connected layer, output 10 classes
    self.out = nn.Linear(32 * 7 * 7, 10)
    
  def forward_impl(self, x):
    x = self.conv1(x)
    x = self.conv2(x)
    # flatten the output of conv2 to (batch_size, 32 * 7 * 7)
    x = x.view(x.size(0), -1)       
    output = self.out(x)
    # A ModelAbstract returns prediction, features, and secondary output (empty list)
    return output, x, []    

In [ ]:
import torch, ednaml
from ednaml.core import EdnaML
torch.__version__

## 1. Basic MNIST Experiment

We first run our basic MNIST experiment, with the default options.

In [ ]:
EdnaML.clear_registrations()
cfg = "./EdnaML/usage-docs/sample-configs/1-storage/mnist.yml"
eml = EdnaML(config=cfg, config_inject = [
    ("MODEL.MODEL_BASE", "simple"),  
    ("SAVE.MODEL_BACKBONE", "simple"),  
    ("TRANSFORMATION.BATCH_SIZE", 64),   # We will also increase the batch size
    ("LOGGING.INPUT_SIZE", [64,1,28,28]),   # We will also fix the input size
])
eml.cfg.MODEL.MODEL_KWARGS = {}       # We delete the old MODEL_KWARGS, because our new model needs no arguments
eml.addModelClass(MNISTModel)

In [ ]:
# These are the default options.
eml.apply(  storage_manager_mode = "strict",
            storage_mode = "local",
            backup_mode = "hybrid",
            tracking_run = None,
            new_run = False,
            skip_storage = False
          )

In [ ]:
eml.train()

In [ ]:
eml.eval()

## What to note

On the file view (if you are on Colab), you should see a directory called `mnist_resnet-v1-simple-mnist` that contains a directory `0`. Inside this, there should be several pytorch files and log files.

Here, the name of the experiment is `mnist_resnet-v1-simple-mnist`, inherited from the `MODEL_CORE_NAME`, `MODEL_VERSION`, `MODEL_BACKBONE`, and `MODEL_QUALIFIER` in the `SAVE` section of the [configuration, linked here](mnist.yml#L20).

`0` is the run for this experiment. There should be a `metrics.json` with the computed metrics.

## 2. `storage_mode`

We will let `storage_mode="empty"` instead of `local`. Here, no local files will be created. Note: we have incremented the version to `2` in `config_inject`, so if `storage_mode` was local, the expected directory would be `mnist_resnet-v2-simple-mnist`. However, it will not be created.

In [ ]:
EdnaML.clear_registrations()
cfg = "./EdnaML/usage-docs/sample-configs/1-storage/mnist.yml"
eml = EdnaML(config=cfg, config_inject = [
    ("MODEL.MODEL_VERSION", 2),  
    ("MODEL.MODEL_BASE", "simple"),  
    ("SAVE.MODEL_BACKBONE", "simple"),  
    ("TRANSFORMATION.BATCH_SIZE", 64),   # We will also increase the batch size
    ("LOGGING.INPUT_SIZE", [64,1,28,28]),   # We will also fix the input size
])
eml.cfg.MODEL.MODEL_KWARGS = {}       # We delete the old MODEL_KWARGS, because our new model needs no arguments
eml.addModelClass(MNISTModel)

In [ ]:
# These are the default options.
eml.apply(  storage_manager_mode = "strict",
            storage_mode = "empty",
            backup_mode = "hybrid",
            tracking_run = None,
            new_run = False,
            skip_storage = False
          )

In [ ]:
eml.train()

In [ ]:
eml.eval()

## 3. `new_run`

Each EdnaML experiment can have multiple runs. Runs are denoted by integers starting from 0. When `new_run` is set to `True`, the runs are incremented. Here, we will switch back to version 1 of the experiment. Inside the directory for experiment 1 (`mnist_resnet-v1-simple-mnist`), we will get 2 new directories: `0` and `1`.

In [ ]:
EdnaML.clear_registrations()
cfg = "./EdnaML/usage-docs/sample-configs/1-storage/mnist.yml"
eml = EdnaML(config=cfg, config_inject = [
    ("MODEL.MODEL_VERSION", 1),  
    ("MODEL.MODEL_BASE", "simple"),  
    ("SAVE.MODEL_BACKBONE", "simple"),  
    ("TRANSFORMATION.BATCH_SIZE", 64),   # We will also increase the batch size
    ("LOGGING.INPUT_SIZE", [64,1,28,28]),   # We will also fix the input size
])
eml.cfg.MODEL.MODEL_KWARGS = {}       # We delete the old MODEL_KWARGS, because our new model needs no arguments
eml.addModelClass(MNISTModel)

In [ ]:
# These are the default options.
eml.apply(  storage_manager_mode = "strict",
            storage_mode = "local",
            backup_mode = "hybrid",
            tracking_run = None,
            new_run = True,
            skip_storage = False
          )

In [ ]:
eml.train()

In [ ]:
eml.eval()

## 4. `tracking_run`

Each experiment can have multiple runs. `tracking_run` selects a specific run to continue from, if there are multiple, or to force a specific run. 

If `tracking_run=None` and `new_run=False`, EdnaML will automatically switch to the most recent run. Here, we will go to version 1 of the experiment, and track run 0 instead of automatically following run 1.

We will also increment number of execution epochs to 2, so we can continue training from where we left off. 



In [ ]:
EdnaML.clear_registrations()
cfg = "./EdnaML/usage-docs/sample-configs/1-storage/mnist.yml"
eml = EdnaML(config=cfg, config_inject = [
    ("MODEL.MODEL_VERSION", 1),  
    ("EXECUTION.EPOCHS", 2),
    ("MODEL.MODEL_BASE", "simple"),  
    ("SAVE.MODEL_BACKBONE", "simple"),  
    ("TRANSFORMATION.BATCH_SIZE", 64),   # We will also increase the batch size
    ("LOGGING.INPUT_SIZE", [64,1,28,28]),   # We will also fix the input size
])
eml.cfg.MODEL.MODEL_KWARGS = {}       # We delete the old MODEL_KWARGS, because our new model needs no arguments
eml.addModelClass(MNISTModel)

In [ ]:
# These are the default options.
eml.apply(  storage_manager_mode = "strict",
            storage_mode = "local",
            backup_mode = "hybrid",
            tracking_run = 0,
            new_run = False,
            skip_storage = False
          )

In [ ]:
eml.train()

In [ ]:
eml.eval()

## 5. Using Storage for backup

So far, we have not performed backups, and instead used the local storage to record artifacts.

Here we will set up a backup for our models. To keep things simple, we will use an `ednaml.storage.LocalStorage` instance, which simple backs up to a user-defined directory on the same machine. LocalStorage can be used to keep redundant copies, or to transfer models to a NAS or some mapped network drive. For example, if running on Google Colab, one can mount their Google Drive at `/content/drive` and use LocalStorage pointing to `/content/drive` to back up artifacts to Google Drive.

In our case, we will backup all artifacts to a directory called `backup`. The Storage options are  provided in `mnist_simple_storage.yml` and will be chained with `mnist.yml`.

We will use version 1, with tracking run 3.

In [ ]:
EdnaML.clear_registrations()
cfg = "./EdnaML/usage-docs/sample-configs/1-storage/mnist.yml"
storage_cfg = "./EdnaML/usage-docs/sample-configs/1-storage/mnist_simple_storage.yml"
eml = EdnaML(config=[cfg, storage_cfg], config_inject = [
    ("MODEL.MODEL_VERSION", 1),  
    ("MODEL.MODEL_BASE", "simple"),  
    ("SAVE.MODEL_BACKBONE", "simple"),  
    ("TRANSFORMATION.BATCH_SIZE", 64),   # We will also increase the batch size
    ("LOGGING.INPUT_SIZE", [64,1,28,28]),   # We will also fix the input size
])
eml.cfg.MODEL.MODEL_KWARGS = {}       # We delete the old MODEL_KWARGS, because our new model needs no arguments
eml.addModelClass(MNISTModel)

In [ ]:
# These are the default options.
eml.apply(  storage_manager_mode = "strict",
            storage_mode = "local",
            backup_mode = "hybrid",
            tracking_run = 3,
            new_run = False,
            skip_storage = False
          )

In [ ]:
eml.train()

In [ ]:
eml.eval()

## 6. Disabling Storage for backup

We can also disable any storages we have set up, if we wanted only local storage. We do this by passing `skip_storage=True`.

Here, we will use a new run, 4. We will also construct the storages by chaining `mnist_simple_storage.yml`. Finally, we will skip constructing the storage.

In [ ]:
EdnaML.clear_registrations()
cfg = "./EdnaML/usage-docs/sample-configs/1-storage/mnist.yml"
storage_cfg = "./EdnaML/usage-docs/sample-configs/1-storage/mnist_simple_storage.yml"
eml = EdnaML(config=[cfg, storage_cfg], config_inject = [
    ("MODEL.MODEL_VERSION", 1),  
    ("MODEL.MODEL_BASE", "simple"),  
    ("SAVE.MODEL_BACKBONE", "simple"),  
    ("TRANSFORMATION.BATCH_SIZE", 64),   # We will also increase the batch size
    ("LOGGING.INPUT_SIZE", [64,1,28,28]),   # We will also fix the input size
])
eml.cfg.MODEL.MODEL_KWARGS = {}       # We delete the old MODEL_KWARGS, because our new model needs no arguments
eml.addModelClass(MNISTModel)

In [ ]:
# These are the default options.
eml.apply(  storage_manager_mode = "strict",
            storage_mode = "local",
            backup_mode = "hybrid",
            tracking_run = 4,
            new_run = False,
            skip_storage = True
          )

In [ ]:
eml.train()

In [ ]:
eml.eval()

## 7. `backup_mode`

`backup_mode` controls how each artifact is backed up. An artifact can be backed up in 2 modes:

1. `canonical` mode: There is one file for the artifact. Each time an artifact is saved, we override the single `canonical` key for the artifact.
2. `ers` mode: EdnaML uses the notion of `<Experiment-Run-Storage>`, or ERS to track every artifact. Additional details can be found in []. In `ers`, a file is created in the remote storage every FREQUENCY and FREQUENCY_STEP.


For local saves, EdnaML always uses `ers` mode. You will notice files with the pattern [artifact]_epoch[num]_step[num].[ext]. We can control the backup mode with `backup_mode`. If `backup_mode=ers`, every artifact is saved in `ers` mode. If `backup_mode=canonical`, every artifact is saved in `canonical` mode. 

If `backup_mode=hybrid`, EdnaML uses intelligent options: models and training artifacts are saved in `ers` mode, while plugins, metrics, logs, and configs are saved in `canonical` mode. You can see this in prior experiments where the backups have a single log file: `log.log`, while the local files contain multiple log files.

Here, we will set `backup_mode` to `canonical` in run 5. Usually, one should not mix up `backup_modes` between runs of the same experiment, but for the purposes of this notebook, it suffices.

In [ ]:
EdnaML.clear_registrations()
cfg = "./EdnaML/usage-docs/sample-configs/1-storage/mnist.yml"
storage_cfg = "./EdnaML/usage-docs/sample-configs/1-storage/mnist_simple_storage.yml"
eml = EdnaML(config=[cfg, storage_cfg], config_inject = [
    ("MODEL.MODEL_VERSION", 1),  
    ("MODEL.MODEL_BASE", "simple"),  
    ("SAVE.MODEL_BACKBONE", "simple"),  
    ("TRANSFORMATION.BATCH_SIZE", 64),   # We will also increase the batch size
    ("LOGGING.INPUT_SIZE", [64,1,28,28]),   # We will also fix the input size
])
eml.cfg.MODEL.MODEL_KWARGS = {}       # We delete the old MODEL_KWARGS, because our new model needs no arguments
eml.addModelClass(MNISTModel)

In [ ]:
# These are the default options.
eml.apply(  storage_manager_mode = "strict",
            storage_mode = "local",
            backup_mode = "canonical",
            tracking_run = 5,
            new_run = False,
            skip_storage = False
          )

In [ ]:
eml.train()

In [ ]:
eml.eval()

## 8. Multiple Storages

Here, we will use multiple storages, and fine-tune backups, with run 6.

Specifically, we will save models to a `LocalStorage` pointing to `./backup`. We will save artifacts to a `LocalStorage` pointing to `./artifacts`. We will skip saving logs.

In [ ]:
EdnaML.clear_registrations()
cfg = "./EdnaML/usage-docs/sample-configs/1-storage/mnist.yml"
storage_cfg = "./EdnaML/usage-docs/sample-configs/1-storage/mnist_multiple_storage.yml"
eml = EdnaML(config=[cfg, storage_cfg], config_inject = [
    ("MODEL.MODEL_VERSION", 1),  
    ("MODEL.MODEL_BASE", "simple"),  
    ("SAVE.MODEL_BACKBONE", "simple"),  
    ("TRANSFORMATION.BATCH_SIZE", 64),   # We will also increase the batch size
    ("LOGGING.INPUT_SIZE", [64,1,28,28]),   # We will also fix the input size
])
eml.cfg.MODEL.MODEL_KWARGS = {}       # We delete the old MODEL_KWARGS, because our new model needs no arguments
eml.addModelClass(MNISTModel)

In [ ]:
# These are the default options.
eml.apply(  storage_manager_mode = "strict",
            storage_mode = "local",
            backup_mode = "hybrid",
            tracking_run = 6,
            new_run = False,
            skip_storage = False
          )

In [ ]:
eml.train()

In [ ]:
eml.eval()

## 9. Deployment

Once an experiment is finished, we can deploy it to rerun it on the same training data, or on new data.

Here, we use EdnaDeploy instead of EdnaML. EdnaDeploy does not build any training components, such as optimizer, scheduler, or loss functions. Furthermore, EdnaDeploy uses a Deployment instead of a Trainer class to manage the pipeline. As such, any Trainer should have a corresponding Deployment object (but not necessarily vice versa).

Here, we will rerun Run 2.

In [ ]:
from ednaml.core import EdnaDeploy
from ednaml.deploy import ClassificationDeploy

In [ ]:
EdnaDeploy.clear_registrations()
cfg = "./EdnaML/usage-docs/sample-configs/1-storage/mnist.yml"
deploy_cfg = "./EdnaML/usage-docs/sample-configs/1-storage/mnist-deploy.yml"
ed = EdnaDeploy(config=[cfg, deploy_cfg], config_inject = [
    ("SAVE.MODEL_VERSION", 1),            # We switch to version 3 for this experiment
    ("MODEL.MODEL_BASE", "simple"),            # We switch to version 3 for this experiment
    ("TRANSFORMATION.BATCH_SIZE", 64),   # We will also increase the batch size
    ("LOGGING.INPUT_SIZE", [256,1,28,28]),   # We will also fix the input size
]
)
ed.cfg.MODEL.MODEL_KWARGS = {}       # We delete the old MODEL_KWARGS, because our new model needs no arguments
ed.addModelClass(MNISTModel)
ed.addDeploymentClass(ClassificationDeploy)

In [ ]:
ed.apply(tracking_run=3, new_run=False)

In [ ]:
ed.deploy()

## 10. Deployment from specific epoch-step pair

Now we will evaluate a specific epoch-step pair from Run 3. Since we want a specific epoch-step pair, and not the latest, we will have to tell EdnaDeploy, in `apply()` to skip loading the latest weights, with `skip_weights=True`

In [ ]:
from ednaml.core import EdnaDeploy
from ednaml.deploy import ClassificationDeploy

In [ ]:
EdnaDeploy.clear_registrations()
cfg = "./EdnaML/usage-docs/sample-configs/1-storage/mnist.yml"
deploy_cfg = "./EdnaML/usage-docs/sample-configs/1-storage/mnist-deploy.yml"
ed = EdnaDeploy(config=[cfg, deploy_cfg], config_inject = [
    ("SAVE.MODEL_VERSION", 1),            # We switch to version 3 for this experiment
    ("MODEL.MODEL_BASE", "simple"),            # We switch to version 3 for this experiment
    ("TRANSFORMATION.BATCH_SIZE", 64),   # We will also increase the batch size
    ("LOGGING.INPUT_SIZE", [256,1,28,28]),   # We will also fix the input size
]
)
ed.cfg.MODEL.MODEL_KWARGS = {}       # We delete the old MODEL_KWARGS, because our new model needs no arguments
ed.addModelClass(MNISTModel)
ed.addDeploymentClass(ClassificationDeploy)

In [ ]:
ed.apply(tracking_run=3, new_run=False, skip_weights=True)

In [ ]:
ed.deploy(continue_epoch = 0, continue_step = 600)

## 11. Deployment on new data

Finally, we will run our MNIST model on CIFAR. 

We need to make a few changes:

1. CIFAR is a 3-channel dataset. MNIST is 1 channel. We will adjust the CIFAR data during dataloading to be 1-channel by passing an argument for GENERATOR_KWARGS, by passing a grayscale image
2. We will overwrite the data loading parts with a CIFAR-10 configuration

In [ ]:
from ednaml.core import EdnaDeploy
from ednaml.deploy import ClassificationDeploy

In [ ]:
EdnaDeploy.clear_registrations()
cfg = "./EdnaML/usage-docs/sample-configs/1-storage/mnist.yml"
deploy_cfg = "./EdnaML/usage-docs/sample-configs/1-storage/mnist-deploy.yml"
cifar10_data = "./EdnaML/usage-docs/sample-configs/1-storage/cifar10-data.yml"
ed = EdnaDeploy(config=[cfg, deploy_cfg, cifar10_data], config_inject = [
    ("SAVE.MODEL_VERSION", 1),            # We switch to version 3 for this experiment
    ("MODEL.MODEL_BASE", "simple"),            # We switch to version 3 for this experiment
    ("TRANSFORMATION.BATCH_SIZE", 64),   # We will also increase the batch size
    ("LOGGING.INPUT_SIZE", [256,1,28,28]),   # We will also fix the input size
]
)
ed.cfg.MODEL.MODEL_KWARGS = {}       # We delete the old MODEL_KWARGS, because our new model needs no arguments
ed.addModelClass(MNISTModel)
ed.addDeploymentClass(ClassificationDeploy)

In [ ]:
ed.apply(tracking_run=3, new_run=False)

In [ ]:
ed.deploy()

# Separator